# TP: Text segmentation using Hidden Markov Models
## SD-TSIA214 | Télécom-Paris 2024
### Carlos Gruss

### Q.1. Give the value of the $\pi$ vector of the initial probabilities:

The initial probabilities vector $\pi$ will be:

$$\pi = \begin{bmatrix} 1 \\ 0 \end{bmatrix}$$

This is beacuse the first state is always the start state (we assume we always start with a header and proceed to the body).

### Q.2. : What is the probability to move from state 1 to state 2  What is the probability to remain in state 2? What is the lower/higher probability? Try to explain why.

The transition matrix $A$ is:

$$A = \begin{pmatrix} 0.999218078035812 & 0.000781921964187974 \\ 0 & 1 \end{pmatrix}$$

For every character in the header, the probability that the next character is in the header (state 1 -> state 1) is $0.999218078035812$. This is because headers contain a lot of characters and only one of them is the last one (the character that transitions from state 1 to state 2).

The probability to move from state 2 to state 1 is of course 0, because once we are in the body of the email it is impossible to go back to the header. Hence the probability to remain in state 2 is 1.

### Q.3. What is the size of B?

The observation probability matrix $B$ is given by:

$$
(B)_{i,j} = P(\text{symbol i} \,\, | \,\, \text{state j})
$$

So the matrix $B$ has as many rows as possible symbols, $N$, and columns as states (two). 

$$
B \in \mathcal{M}^{N \times 2}
$$

In [1]:
import os
import glob
import numpy as np

In [83]:
ROOT = os.path.abspath('.')

PERL_DIR = os.path.join(ROOT, 'PerlScriptAndModel')
RES_DIR = os.path.join(ROOT, 'res')
DATA_DIR = os.path.join(ROOT, 'dat')
SEG_DIR = os.path.join(ROOT, 'seg')

### Coding/Decoding Mails

In [3]:
# Iterate through files and load the text 
def files_iter(data_dir, with_name=False):
    files = glob.glob('{}/*.dat'.format((data_dir)))
    if with_name:
        for f in files:
            # Get the filename 
            name = f.split('/')[-1].split('.')[0]
            # Return filename and associated text
            yield name, np.loadtxt(f)
    else:
        for f in files :
            yield np.loadtxt(f)

In [4]:
#  And we get a generator that will allow us to iterate through the mails
mail_iter = list(files_iter(DATA_DIR, with_name=True))

In [5]:
# Count the emails in mail_iter
n_mails = sum([1 for _ in mail_iter])
print('Number of emails: {}'.format(n_mails))
print('First email: \n{}'.format(mail_iter[0]))

Number of emails: 30
First email: 
('c:\\Users\\carlos\\Downloads\\GitHub\\telecom-sd-tsia214-hmm\\dat\\mail1', array([ 70., 114., 111., ..., 115.,  10.,  10.]))


### Distribution files

In [6]:
# Writing a function to get the probability data
def get_emission_prob(perl_dir):
    return np.loadtxt(os.path.join(perl_dir, 'P.txt'))

In [7]:
# Inputs to the Viterbi function
trans = np.array([[0.999218078035812, 0.000781921964187974], [0, 1]])
emission_prob = get_emission_prob(PERL_DIR)
states = np.array([1, 2])
start_prob = np.array([1, 0])

### To implement:

In [8]:
#  Viterbi function
def viterbi(obs, states, start_prob, trans, emission_prob):
    """
    Viterbi Algorithm Implementation

    Keyword arguments:
        - obs: sequence of observation
        - states: list of states
        - start_prob: vector of the initial probabilities
        - trans: transition matrix
        - emission_prob: emission probability matrix
    Returns:
        - seq: sequence of state
    """

    # Avoid underflow: use the logarithm !
    # Avoid 0 in logarithm: use a small constant !
    small = np.finfo(np.float64).tiny

    start_prob = np.log(start_prob + small)
    trans = np.log(trans + small)
    emission_prob = np.log(emission_prob + small)

    T = len(obs)  # Number of observations
    N = len(states)  # Number of model states

    # Initialisation
    log_l = np.zeros((T, N))
    bcktr = np.zeros((T, N))

    # Viterbi algorithm
    # Forward loop:
    log_l[0, :] = start_prob + emission_prob[obs[0], :]
    for t in range(1, T):
        log_l[t, :] = [np.max(log_l[t-1, :] + trans[:, i] + emission_prob[obs[t], i]) for i in range(N)]
        bcktr[t, :] = [np.argmax(log_l[t-1, :] + trans[:, i] + emission_prob[obs[t], i]) for i in range(N)]
    
    # Backward loop
    path = np.zeros(T, dtype=int)
    path[-1] = np.argmax(log_l[-1, :])
    for i in range(T - 1, 0, -1):
        path[i - 1] = bcktr[i, path[i]]

    return [states[i] for i in path]

In [9]:
# Creating a directory to put the result of the viterbi function
if not os.path.exists(RES_DIR):
    os.mkdir(RES_DIR)
    
# Function that will write a viterbi path for a mail in a dedicated result file
def create_viterbi_path_file(mail_name, viterbi_path):
    #print('Path to file: {}/{}_path.txt'.format(RES_DIR, mail_name))
    with open('{}\{}_path.txt'.format(RES_DIR, mail_name), 'w') as f: 
        f.write(''.join([str(c) for c in viterbi_path]))   

In [10]:
# Using our generator, we get the mail names and data
for name_file, data in mail_iter:
    # Turn data into int
    data = [int(d) for d in data]
    # Extract file name from path
    name_file = name_file.split('\\')[-1]
    # Find out the viterbi path using viterbi
    viterbi_path = viterbi(data, states, start_prob, trans, emission_prob)
    # Put it in the result file
    #print('Creating file for {}'.format(name_file))
    #print('Viterbi path: {}'.format(viterbi_path))  
    create_viterbi_path_file(name_file, viterbi_path)

### Visualizing segmentation

In [85]:
def visualize_segmentation(mail_filename, visualized_mail_filename, path):
    ## @parameter mail_filename : Path to the mail on wich we try the algorithm.
    ## @parameter visualized_mail_filename : The path on which we write the mail with the v 	
    ## @parameter path : The sequence of 0 and 1 that the Viterbi algorithm returns.
    ## return: True if the header corresponds to the
    i=0
    while path[i] == 0:
        i+=1
        visu = open(visualized_mail_filename, 'w') 
        if(mail_filename.endswith(".dat")):
            mail_filename = mail_filename[:-4] + ".txt" 
        mail = open(mail_filename, 'r')
        header = mail.read(i)
        visu.write(header) 
        visu.write("\n===================== cut here\n") 
        visu.write(mail.read(path.sum()))
        visu.close() 
        mail.close() 
    return

# Writing a function getting the original mail, the result of viterbi, and applying the segmentation script
# Then putting the result
def segment_mail(mail_name, data_dir, output_dir):
    # Get the full path of the mail
    path_mail = os.path.join(data_dir, mail_name + '.dat')
    # Get the full path of the result
    path_res = os.path.join(output_dir, mail_name + '_path.txt')
    # Segment the mail
    with open(path_res, 'r') as file:
        numbers = file.read().strip() # This assumes your numbers are in a continuous sequence without spaces
        viterbi_path = [int(digit) for digit in numbers]
    mail_text = np.loadtxt(path_mail)

    print('Segmenting mail {}'.format(mail_name))

    header = ''.join([chr(int(mail_text[i])) for i in range(len(viterbi_path)) if viterbi_path[i] == 1])
    body = ''.join([chr(int(mail_text[i])) for i in range(len(viterbi_path)) if viterbi_path[i] == 2])

    return header, body, (not body)

In [86]:
# Getting mails names
mail_paths = [name for name, _ in files_iter(DATA_DIR, with_name=True)]
mail_names = [name.split('\\')[-1].split('.')[0] for name in mail_paths]
# Call the function and look at the result of segmentation
header, body, body_empty = segment_mail(mail_names[0], DATA_DIR, RES_DIR)

if not body_empty:
    print('Mail segmented succesfully: \n')
    print('=====================================\n')
    print('Header: \n\n{}'.format(header))
    print('\n=====================================\n')
    print('Body: \n\n{}'.format(body))
else:
    print('Mail not segmented: \n')
    print('\n=====================================\n')
    print('Mail: \n\n{}'.format(header))

Segmenting mail mail1
Mail segmented succesfully: 


Header: 

From exmh-workers-admin@redhat.com  Thu Aug 22 12:36:23 2002
Return-Path: <exmh-workers-admin@spamassassin.taint.org>
Delivered-To: zzzz@localhost.netnoteinc.com
Received: from localhost (localhost [127.0.0.1])
	by phobos.labs.netnoteinc.com (Postfix) with ESMTP id D03E543C36
	for <zzzz@localhost>; Thu, 22 Aug 2002 07:36:16 -0400 (EDT)
Received: from phobos [127.0.0.1]
	by localhost with IMAP (fetchmail-5.9.0)
	for zzzz@localhost (single-drop); Thu, 22 Aug 2002 12:36:16 +0100 (IST)
Received: from listman.spamassassin.taint.org (listman.spamassassin.taint.org [66.187.233.211]) by
    dogma.slashnull.org (8.11.6/8.11.6) with ESMTP id g7MBYrZ04811 for
    <zzzz-exmh@spamassassin.taint.org>; Thu, 22 Aug 2002 12:34:53 +0100
Received: from listman.spamassassin.taint.org (localhost.localdomain [127.0.0.1]) by
    listman.redhat.com (Postfix) with ESMTP id 8386540858; Thu, 22 Aug 2002
    07:35:02 -0400 (EDT)
Delivered-To: exmh-wor

In [88]:
# Creating a directory to put the result of segmentation
if not os.path.exists(SEG_DIR):
    os.mkdir(SEG_DIR)

# Writing a function to put the result of the segmentation in a file
def write_segmented_mail(mail_name, header, body, output_dir):
    with open('{}\{}.txt'.format(output_dir, mail_name), 'w', encoding="utf-8") as f:
        f.write('Header: \n\n{}'.format(header))
        f.write('\n=====================================\n')
        f.write('Body: \n\n{}'.format(body))

# Using our generator, we get the mail names and data
for name_file, _ in mail_iter:
    # Extract file name from path
    name_file = name_file.split('\\')[-1]
    # Find out the viterbi path using viterbi
    header, body, body_empty = segment_mail(name_file, DATA_DIR, RES_DIR)
    # Put it in the result file
    if not body_empty:
        write_segmented_mail(name_file, header, body, SEG_DIR)
    else:
        with open('{}\{}.txt'.format(SEG_DIR, name_file), 'w') as f:
            f.write('Mail not segmented: \n')
            f.write('\n=====================================\n')
            f.write('Mail: \n\n{}'.format(header))



Segmenting mail mail1
Segmenting mail mail10
Segmenting mail mail11
Segmenting mail mail12
Segmenting mail mail13
Segmenting mail mail14
Segmenting mail mail15
Segmenting mail mail16
Segmenting mail mail17
Segmenting mail mail18
Segmenting mail mail19
Segmenting mail mail2
Segmenting mail mail20
Segmenting mail mail21
Segmenting mail mail22
Segmenting mail mail23
Segmenting mail mail24
Segmenting mail mail25
Segmenting mail mail26
Segmenting mail mail27
Segmenting mail mail28
Segmenting mail mail29
Segmenting mail mail3
Segmenting mail mail30
Segmenting mail mail4
Segmenting mail mail5
Segmenting mail mail6
Segmenting mail mail7
Segmenting mail mail8
Segmenting mail mail9
